In [1]:
import moviepy.video.io.VideoFileClip as mpy
import cv2
import numpy as np
vid = mpy.VideoFileClip("shapes_video.mp4")
framecount = vid.reader.nframes
print(vid.size[0]-1,vid.size[1]-1)

videofps = vid.fps
images_list =[]
#Since we are dealing with salt&pepper, I implemented an order statics filter.

for i in range(framecount):
    frame = vid.get_frame(i*1.0/ videofps)
    temp = np.empty((frame.shape[0],frame.shape[1],3), np.uint8)
    #temp is the new frame that without noise
    for r in range(1,(frame.shape[0]-1)):
        for c in range(1,(frame.shape[1]-1)):
            img_new = sorted([frame[r-1, c-1][0],frame[r-1, c][0], frame[r-1, c+ 1][0],frame[r, c-1][0],
                        frame[r, c][0], frame[r, c + 1][0], frame[r + 1, c-1][0], frame[r + 1, c][0],
                        frame[r + 1, c + 1][0]])
            temp[r, c]= img_new[4]
            #sort 3x3 window of pixel, and take the middle one-> median filter
    temp = temp.astype(np.uint8)
    images_list.append(temp)            


719 575


In [2]:
import moviepy.video.io.ImageSequenceClip as m

clip = m.ImageSequenceClip(images_list, fps = 25)
clip.write_videofile("hw2_part1_video.mp4", codec="libx264")

t:  15%|██████████▏                                                         | 14/93 [00:00<00:00, 137.59it/s, now=None]

Moviepy - Building video hw2_part1_video.mp4.
Moviepy - Writing video hw2_part1_video.mp4



Moviepy - Done !
Moviepy - video ready hw2_part1_video.mp4


## PART 1.2

In [8]:
def corner_detector(image): 
    return_img=image.copy()
    
    image = np.mean(cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT), axis=-1)
    # It stills see some noice since corner detector is very sensitive for noise. So I implemented gaussian first.
        
    dx,dy=np.gradient(image)

    I_xx=dx*dx
    I_xy=dx*dy
    I_yy=dy*dy    
    G_matrix=[[I_xx,I_xy],
              [I_xy,I_yy]]
    #not the final version of image structure tensor    
    
    counter=0
    for row in range(1, image.shape[0] - 1):
        for col in range(1, image.shape[1] -1):
            G_matrix[0][0] = I_xx[row - 1 : row + 1, col - 1 : col + 1]
            G_matrix[0][1] = I_xy[row - 1 : row + 1, col - 1 : col + 1]
            G_matrix[1][0] = G_matrix[0][1]
            G_matrix[1][1] = I_yy[row - 1 : row + 1, col - 1 : col + 1]

            #formula from slide-week7.52
            
            structure_tensor = [[np.sum(G_matrix[0][0]), np.sum(G_matrix[0][1])],
                                [np.sum(G_matrix[1][0]), np.sum(G_matrix[1][1])]]
            eigenvals = np.linalg.eigvals(np.array(structure_tensor))
            min_eigenval = np.min(eigenvals)
            #threshold=np.linalg.det(structure_tensor)-(0.05*np.trace(structure_tensor))         
            
            #I tried 10000,1000,500->large,100->little
            if min_eigenval > 200: #a treshold                
                counter+=1
                return_img[row-1:row+1, col-1:col+1] = (255,0,0)
                #found a corner
    
    return counter,return_img

In [16]:
import moviepy.video.io.VideoFileClip as mpy
import cv2
import numpy as np
vid = mpy.VideoFileClip("hw2_part1_video.mp4")

framecount = vid.reader.nframes
videofps = vid.fps
background=np.empty((vid.size[1],vid.size[0],3), np.uint8)
star=0
square=0
pentagon=0
images_list=[]
for i in range(framecount):
    frame = vid.get_frame(i*1.0/ videofps)
    img = frame.copy()
    img.setflags(write=1)
    
    if(i>6 and i <16): #first 6 frame has no shape 
                       #after 16th frame, shapes intersects and whites become black. I could not solve it
        img =cv2.subtract(frame,background)
        
        background+=img.copy()        
        counter,return_img = corner_detector(img)
        images_list.append(return_img)

In [18]:
import moviepy.video.io.ImageSequenceClip as m

clip = m.ImageSequenceClip(images_list, fps = 5)
clip.write_videofile("hw2_part1_2_video.mp4", codec="libx264")

Moviepy - Building video hw2_part1_2_video.mp4.
Moviepy - Writing video hw2_part1_2_video.mp4

Moviepy - Done !
Moviepy - video ready hw2_part1_2_video.mp4


I could not able the count exact corner, so I left this part.

In [19]:
for i in range (len(images_list)):
    cv2.imwrite("shape_"+str(i)+".png",images_list[i])